In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
! unzip -q '/content/drive/My Drive/tiny-imagenet-200.zip'

In [3]:
%cd /content/drive/My\ Drive/Assignment\ 4/

/content/drive/My Drive/Assignment 4


In [4]:
# Utilities

import os
import cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

TRAIN = "/content/tiny-imagenet-200/train/"
VAL = "/content/tiny-imagenet-200/val/images"
VAL_ANNOT = "/content/tiny-imagenet-200/val/val_annotations.txt"
WORDNET = "/content/tiny-imagenet-200/wnits.txt"
WORD_LABELS = "/content/tiny-imagenet-200/words.txt"


Using TensorFlow backend.


In [16]:
# Load the data

from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from imgaug import augmenters as iaa

val_data = pd.read_csv(VAL_ANNOT , sep='\t', names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X','Y','H', 'W'], axis=1, inplace=True)

# seq = iaa.Sequential([iaa.Pepper(0.02)])
# Augmentations to be include: Increased Contrast, Random Erasing/Cutout/Occlusion, SalandPepper Noise (Prevents Overfitting)
# rescale=1./255, rescale=1./255

train_datagen = ImageDataGenerator(
        rotation_range=18, 
        zoom_range=0.15,
        width_shift_range=0.2, 
        height_shift_range=0.2, 
        shear_range=0.15,
        brightness_range=[0.4, 1.6],
        horizontal_flip=True, 
        zca_whitening=True,
        zca_epsilon=1e-06,
        fill_mode="reflect"
)

# preprocessing_function=(seq.augment_image)

train_generator = train_datagen.flow_from_directory(
        TRAIN,
        target_size=(64, 64),
        batch_size=64,
        class_mode='categorical')

val_datagen = ImageDataGenerator()

val_generator = val_datagen.flow_from_dataframe(
    val_data, directory=VAL, 
    x_col='File', 
    y_col='Class', 
    target_size=(64, 64),
    color_mode='rgb', 
    class_mode='categorical', 
    batch_size=64, 
    shuffle=False, 
    seed=42
)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:334: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
# Build the model

from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, SeparableConv2D
from keras.layers.convolutional import AveragePooling2D, MaxPooling2D, ZeroPadding2D
from keras.layers.core import Activation, Dense
from CyclicLearningRate.clr_callback import *
from keras.layers import Flatten, Input, add, GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras.callbacks import *
from keras.models import Model
from keras.regularizers import l2
from keras import backend as K

class ResNet:
	@staticmethod
	def residual_module(data, K, stride, chanDim, red=False, reg=0.0001, bnEps=2e-5, bnMom=0.9):
		shortcut = data

		bn1 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom, beta_initializer="zeros", gamma_initializer="ones")(data)
		act1 = Activation("relu")(bn1)
		conv1 = Conv2D(int(K * 0.25), (1, 1), use_bias=False, kernel_regularizer=l2(reg))(act1)

		bn2 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom, beta_initializer="zeros", gamma_initializer="ones")(conv1)
		act2 = Activation("relu")(bn2)
		conv2 = SeparableConv2D(int(K * 0.25), (3, 3), strides=stride, padding="same", use_bias=False, depthwise_regularizer=l2(reg), depthwise_initializer='glorot_uniform')(act2)

		bn3 = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom, beta_initializer="zeros", gamma_initializer="ones")(conv2)
		act3 = Activation("relu")(bn3)
		conv3 = Conv2D(K, (1, 1), use_bias=False, kernel_regularizer=l2(reg))(act3)

		if red and stride == (2,2):
			shortcut = AveragePooling2D((2,2))(bn1)

		shortcut = Conv2D(K, (1,1))(shortcut)
		x = add([conv3, shortcut])

		return x

	@staticmethod
	def build(width, height, depth, classes, stages, filters, reg=0.0001, bnEps=2e-5, bnMom=0.9):
		inputShape = (height, width, depth)
		chanDim = -1

		inputs = Input(shape=inputShape)
		x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom, beta_initializer="zeros", gamma_initializer="ones")(inputs)
		x = Activation("relu")(x)
		x = SeparableConv2D(filters[0], (5, 5), use_bias=False, padding="same", depthwise_regularizer=l2(reg), depthwise_initializer='glorot_uniform')(x)
		x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom, beta_initializer="zeros", gamma_initializer="ones")(x)
		x = Activation("relu")(x)
		x = ZeroPadding2D((1, 1))(x)
		x = MaxPooling2D((3, 3), strides=(2, 2))(x)
    
		for i in range(0, len(stages)):
			stride = (1, 1) if i == 0 else (2, 2)
			x = ResNet.residual_module(x, filters[i + 1], stride, chanDim, red=True, bnEps=bnEps, bnMom=bnMom)

			for j in range(0, stages[i] - 1):
				x = ResNet.residual_module(x, filters[i + 1], (1, 1), chanDim, bnEps=bnEps, bnMom=bnMom)

		x = BatchNormalization(axis=chanDim, epsilon=bnEps, momentum=bnMom)(x)
		x = Activation("relu")(x)
		x = Conv2D(200, (1,1), kernel_regularizer=l2(reg))(x)
		x = GlobalAveragePooling2D('channels_last')(x)
		# x = Flatten()(x)
		x = Activation("softmax")(x)

		model = Model(inputs, x, name="resnet")

		return model

In [0]:
from keras.callbacks import Callback

class EpochCheckpoint(Callback):
	def __init__(self, outputPath, every=5, startAt=0):
		# call the parent constructor
		super(Callback, self).__init__()
    
		self.outputPath = outputPath
		self.every = every
		self.intEpoch = startAt

	def on_epoch_end(self, epoch, logs={}):
		# check to see if the model should be serialized to disk
		if (self.intEpoch + 1) % self.every == 0:
			p = os.path.sep.join([self.outputPath,
				"resnet.hdf5".format(self.intEpoch + 1)])
			self.model.save(p, overwrite=True)
		self.intEpoch += 1
    

In [0]:
# Learning Rate Decay

from keras.callbacks import LearningRateScheduler

NUM_EPOCHS = 35
INIT_LR = 0.0001

def poly_decay(epoch):
  maxEpochs = NUM_EPOCHS
  baseLR = INIT_LR
  power = 1.0
  
  # compute the new learning rate based on polynomial decay
  alpha = baseLR * (1 - (epoch / float(maxEpochs))) ** power
  
  # return the new learning rate
  return alpha

In [0]:
import tensorflow
flag = 0

# Cyclic Learning Rate
clr_triangular = CyclicLR(mode='triangular', max_lr=0.1, base_lr=0.01, step_size=70)

checkpoint_path = "../EIP/boosted_resnet.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback

cp_callback = tensorflow.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_weights_only=False,
                                                 verbose=1, period=5)

callbacks = [
   cp_callback,
    EpochCheckpoint('../EIP/', every=5),
   LearningRateScheduler(poly_decay) 
]
 # clr_triangular

In [0]:
from keras.models import load_model
from keras.optimizers import SGD
# INIT_LR = 0.0000002

if flag == 0: 
  model = ResNet.build(None, None, 3, 200, (3, 4, 6), (64, 128, 256, 512), reg=0.0005)
  opt = SGD(lr=INIT_LR, momentum=0.9, nesterov=True)
  model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

else:
  
  model = load_model('../EIP/main.hdf5')
  
  print(f'Old Learning Rate: {K.get_value(model.optimizer.lr)}')
  K.set_value(model.optimizer.lr, le-3)
  print(f'New Learning Rate: {K.get_value(model.optimizer.lr)}')

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 12          input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
separable_conv2d_1 (SeparableCo (None, None, None, 6 267         activation_1[0][0]               
_____________________________________

### Training on 16x16 images to detect small features

In [0]:
model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 1024,
  validation_data=val_generator,
  validation_steps=10000 // 1024,
  epochs=35,
  max_queue_size=512 * 4,
  callbacks=callbacks, verbose=1
)


Epoch 1/35


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:718: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


97/97 [==============================] - 312s 3s/step - loss: 5.7565 - acc: 0.0120 - val_loss: 5.6483 - val_acc: 0.0196
Epoch 2/35
97/97 [==============================] - 307s 3s/step - loss: 5.5888 - acc: 0.0230 - val_loss: 5.4633 - val_acc: 0.0360
Epoch 3/35
97/97 [==============================] - 302s 3s/step - loss: 5.4245 - acc: 0.0356 - val_loss: 5.3080 - val_acc: 0.0489
Epoch 4/35
97/97 [==============================] - 303s 3s/step - loss: 5.2543 - acc: 0.0506 - val_loss: 5.2082 - val_acc: 0.0541
Epoch 5/35
97/97 [==============================] - 308s 3s/step - loss: 5.0893 - acc: 0.0674 - val_loss: 5.2496 - val_acc: 0.0524

Epoch 00005: saving model to ../EIP/boosted_resnet.ckpt
Epoch 6/35
97/97 [==============================] - 255s 3s/step - loss: 4.9647 - acc: 0.0817 - val_loss: 5.0660 - val_acc: 0.0727
Epoch 7/35
97/97 [==============================] - 311s 3s/step - loss: 4.8408 - acc: 0.0960 - val_loss: 5.0686 - val_acc: 0.0722
Epoch 8/35
97/97 [===================

### Training 32x32 images for 35 epochs

---


### Total epochs = 35

In [0]:
model = load_model(checkpoint_path)
model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 512,
  validation_data=val_generator,
  validation_steps=10000 // 512,
  epochs=35,
  max_queue_size=750,
  callbacks=callbacks, verbose=1
)

Epoch 1/35


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:718: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


195/195 [==============================] - 412s 2s/step - loss: 5.7092 - acc: 0.0166 - val_loss: 5.4648 - val_acc: 0.0278
Epoch 2/35
195/195 [==============================] - 306s 2s/step - loss: 5.3009 - acc: 0.0462 - val_loss: 5.2570 - val_acc: 0.0480
Epoch 3/35
195/195 [==============================] - 200s 1s/step - loss: 4.9521 - acc: 0.0807 - val_loss: 5.4739 - val_acc: 0.0497
Epoch 4/35
195/195 [==============================] - 205s 1s/step - loss: 4.6949 - acc: 0.1107 - val_loss: 4.8513 - val_acc: 0.0895
Epoch 5/35
195/195 [==============================] - 201s 1s/step - loss: 4.4987 - acc: 0.1365 - val_loss: 5.3301 - val_acc: 0.0642

Epoch 00005: saving model to ../EIP/boosted_resnet.ckpt
Epoch 6/35
195/195 [==============================] - 147s 752ms/step - loss: 4.3504 - acc: 0.1552 - val_loss: 4.7256 - val_acc: 0.1097
Epoch 7/35
195/195 [==============================] - 195s 1s/step - loss: 4.2253 - acc: 0.1714 - val_loss: 4.6380 - val_acc: 0.1132
Epoch 8/35
195/195 [

In [0]:
model = load_model(checkpoint_path)
model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 256,
  validation_data=val_generator,
  validation_steps=10000 // 256,
  epochs=15,
  max_queue_size=512,
  callbacks=callbacks, verbose=1
)

Epoch 1/15


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:718: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


  2/390 [..............................] - ETA: 57:48 - loss: 3.1158 - acc: 0.3301  

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.461146). Check your callbacks.
  % delta_t_median)


390/390 [==============================] - 375s 962ms/step - loss: 2.9981 - acc: 0.3680 - val_loss: 4.2515 - val_acc: 0.1653
Epoch 2/15
390/390 [==============================] - 249s 639ms/step - loss: 3.1791 - acc: 0.3327 - val_loss: 4.5715 - val_acc: 0.1261
Epoch 3/15
390/390 [==============================] - 248s 635ms/step - loss: 3.3600 - acc: 0.3010 - val_loss: 4.4114 - val_acc: 0.1396
Epoch 4/15
390/390 [==============================] - 248s 637ms/step - loss: 3.4916 - acc: 0.2801 - val_loss: 4.2428 - val_acc: 0.1551

Epoch 00004: saving model to ../EIP/boosted_resnet.ckpt
Epoch 5/15
390/390 [==============================] - 203s 520ms/step - loss: 3.5558 - acc: 0.2738 - val_loss: 4.4058 - val_acc: 0.1511
Epoch 6/15
390/390 [==============================] - 248s 637ms/step - loss: 3.4468 - acc: 0.2960 - val_loss: 4.2901 - val_acc: 0.1619
Epoch 7/15
390/390 [==============================] - 246s 630ms/step - loss: 3.2831 - acc: 0.3255 - val_loss: 4.1844 - val_acc: 0.1727
Ep

### Completed 50 epochs with 32x32. Will now train it on 64x64

---


### Total epochs = 100

In [0]:
model = load_model(checkpoint_path)
print(f'Old Learning Rate: {K.get_value(model.optimizer.lr)}')
K.set_value(model.optimizer.lr, 0.0005)
print(f'New Learning Rate: {K.get_value(model.optimizer.lr)}')

model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 256,
  validation_data=val_generator,
  validation_steps=10000 // 256,
  epochs=20,
  max_queue_size=512,
  callbacks=callbacks, verbose=1
)

Epoch 1/20


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:718: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


390/390 [==============================] - 516s 1s/step - loss: 3.1358 - acc: 0.3559 - val_loss: 3.8909 - val_acc: 0.2459
Epoch 2/20
390/390 [==============================] - 432s 1s/step - loss: 2.9132 - acc: 0.3985 - val_loss: 3.1467 - val_acc: 0.3601
Epoch 3/20
390/390 [==============================] - 435s 1s/step - loss: 2.7329 - acc: 0.4361 - val_loss: 3.0537 - val_acc: 0.3768
Epoch 4/20
390/390 [==============================] - 433s 1s/step - loss: 2.5348 - acc: 0.4750 - val_loss: 2.9048 - val_acc: 0.4025

Epoch 00004: saving model to ../EIP/boosted_resnet.ckpt
Epoch 5/20
390/390 [==============================] - 428s 1s/step - loss: 2.3232 - acc: 0.5225 - val_loss: 2.7104 - val_acc: 0.4440
Epoch 6/20
390/390 [==============================] - 426s 1s/step - loss: 2.2759 - acc: 0.5325 - val_loss: 2.8030 - val_acc: 0.4256
Epoch 7/20
390/390 [==============================] - 433s 1s/step - loss: 2.4535 - acc: 0.4908 - val_loss: 2.9673 - val_acc: 0.3900
Epoch 8/20
390/390 [===

### Continue training on 64x64 images
---

### Total epochs = 120

In [0]:
model = load_model(checkpoint_path)
model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 256,
  validation_data=val_generator,
  validation_steps=10000 // 256,
  epochs=30,
  max_queue_size=512,
  callbacks=callbacks, verbose=1
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:718: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


390/390 [==============================] - 482s 1s/step - loss: 2.8741 - acc: 0.4134 - val_loss: 3.7208 - val_acc: 0.3193
Epoch 2/30
390/390 [==============================] - 412s 1s/step - loss: 2.8365 - acc: 0.4256 - val_loss: 4.0758 - val_acc: 0.2503
Epoch 3/30
390/390 [==============================] - 415s 1s/step - loss: 2.8042 - acc: 0.4361 - val_loss: 3.3452 - val_acc: 0.3609
Epoch 4/30
390/390 [==============================] - 413s 1s/step - loss: 2.7532 - acc: 0.4485 - val_loss: 3.4295 - val_acc: 0.3228
Epoch 5/30
390/390 [==============================] - 412s 1s/step - loss: 2.7192 - acc: 0.4540 - val_loss: 3.3858 - val_acc: 0.3530

Epoch 00005: saving model to ../EIP/boosted_resnet.ckpt
Epoch 6/30
390/390 [==============================] - 414s 1s/step - loss: 2.6829 - acc: 0.4626 - val_loss: 3.4103 - val_acc: 0.3529
Epoch 7/30
390/390 [==============================] - 413s 1s/step - loss: 2.6495 - acc: 0.4709 - val_loss: 3.2691 - val_acc: 0.3678
Epoch 8/30
390/390 [===

### Trained another 5 epochs to check variablility with CLR

---

### Total epochs  125

In [0]:
model = load_model(checkpoint_path)
model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 256,
  validation_data=val_generator,
  validation_steps=10000 // 256,
  epochs=20,
  max_queue_size=512,
  callbacks=callbacks, verbose=1
)

Epoch 1/20


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:718: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


390/390 [==============================] - 479s 1s/step - loss: 2.5773 - acc: 0.4948 - val_loss: 3.3592 - val_acc: 0.3650
Epoch 2/20
390/390 [==============================] - 403s 1s/step - loss: 2.5754 - acc: 0.5004 - val_loss: 3.9786 - val_acc: 0.3085
Epoch 3/20
390/390 [==============================] - 406s 1s/step - loss: 2.5432 - acc: 0.5091 - val_loss: 3.7231 - val_acc: 0.3319
Epoch 4/20
390/390 [==============================] - 408s 1s/step - loss: 2.5146 - acc: 0.5155 - val_loss: 3.6103 - val_acc: 0.3413
Epoch 5/20
390/390 [==============================] - 408s 1s/step - loss: 2.4828 - acc: 0.5229 - val_loss: 3.3555 - val_acc: 0.3820

Epoch 00005: saving model to ../EIP/boosted_resnet.ckpt
Epoch 6/20
390/390 [==============================] - 407s 1s/step - loss: 2.4555 - acc: 0.5288 - val_loss: 3.0896 - val_acc: 0.4266
Epoch 7/20
390/390 [==============================] - 409s 1s/step - loss: 2.4185 - acc: 0.5356 - val_loss: 3.3363 - val_acc: 0.3973
Epoch 8/20
390/390 [===

### Trained 20 more epochs on 64x64. The accuracies seemed to fluctuate a lot

---

### Total epochs = 145

In [0]:
model = load_model(checkpoint_path)
model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 256,
  validation_data=val_generator,
  validation_steps=10000 // 256,
  epochs=10,
  max_queue_size=512,
  callbacks=callbacks, verbose=1
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:718: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


390/390 [==============================] - 466s 1s/step - loss: 1.8192 - acc: 0.6631 - val_loss: 2.6500 - val_acc: 0.5142
Epoch 2/10
390/390 [==============================] - 400s 1s/step - loss: 1.6436 - acc: 0.7082 - val_loss: 2.5922 - val_acc: 0.5243
Epoch 3/10
390/390 [==============================] - 398s 1s/step - loss: 1.5917 - acc: 0.7219 - val_loss: 2.5882 - val_acc: 0.5251
Epoch 4/10
390/390 [==============================] - 398s 1s/step - loss: 1.5661 - acc: 0.7293 - val_loss: 2.5580 - val_acc: 0.5287
Epoch 5/10
390/390 [==============================] - 398s 1s/step - loss: 1.5473 - acc: 0.7349 - val_loss: 2.5639 - val_acc: 0.5299

Epoch 00005: saving model to ../EIP/boosted_resnet.ckpt
Epoch 6/10
390/390 [==============================] - 397s 1s/step - loss: 1.5258 - acc: 0.7396 - val_loss: 2.5697 - val_acc: 0.5288
Epoch 7/10
390/390 [==============================] - 398s 1s/step - loss: 1.5150 - acc: 0.7413 - val_loss: 2.5780 - val_acc: 0.5287
Epoch 8/10
390/390 [===

### Training on 32x32 images

---

### Total epochs = 155

In [0]:
model = load_model(checkpoint_path)
model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 512,
  validation_data=val_generator,
  validation_steps=10000 // 512,
  epochs=20,
  max_queue_size=512 * 2,
  callbacks=callbacks, verbose=1
)

Epoch 1/20


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:718: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


195/195 [==============================] - 352s 2s/step - loss: 3.7938 - acc: 0.3776 - val_loss: 4.6818 - val_acc: 0.3340
Epoch 2/20
195/195 [==============================] - 341s 2s/step - loss: 3.3375 - acc: 0.4141 - val_loss: 4.4103 - val_acc: 0.3356
Epoch 3/20
195/195 [==============================] - 209s 1s/step - loss: 3.1372 - acc: 0.4259 - val_loss: 4.2588 - val_acc: 0.3386
Epoch 4/20
195/195 [==============================] - 184s 946ms/step - loss: 3.0250 - acc: 0.4362 - val_loss: 4.0927 - val_acc: 0.3440
Epoch 5/20
195/195 [==============================] - 184s 944ms/step - loss: 2.9339 - acc: 0.4448 - val_loss: 3.9718 - val_acc: 0.3491

Epoch 00005: saving model to ../EIP/boosted_resnet.ckpt
Epoch 6/20
195/195 [==============================] - 128s 659ms/step - loss: 2.8773 - acc: 0.4489 - val_loss: 3.8873 - val_acc: 0.3501
Epoch 7/20
195/195 [==============================] - 185s 947ms/step - loss: 2.8334 - acc: 0.4513 - val_loss: 3.8145 - val_acc: 0.3526
Epoch 8/20


### Training on 16x16 images

---

### Total epochs = 175

In [0]:
model = load_model(checkpoint_path)
model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 1024,
  validation_data=val_generator,
  validation_steps=10000 // 1024,
  epochs=20,
  max_queue_size=512 * 4,
  callbacks=callbacks, verbose=1
)

Epoch 1/20


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:718: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


97/97 [==============================] - 282s 3s/step - loss: 5.5741 - acc: 0.1348 - val_loss: 6.0778 - val_acc: 0.1216
Epoch 2/20
97/97 [==============================] - 272s 3s/step - loss: 4.8489 - acc: 0.1434 - val_loss: 5.2318 - val_acc: 0.1256
Epoch 3/20
97/97 [==============================] - 272s 3s/step - loss: 4.5914 - acc: 0.1521 - val_loss: 4.9368 - val_acc: 0.1349
Epoch 4/20
97/97 [==============================] - 272s 3s/step - loss: 4.4888 - acc: 0.1575 - val_loss: 4.8105 - val_acc: 0.1386
Epoch 5/20
97/97 [==============================] - 272s 3s/step - loss: 4.4068 - acc: 0.1654 - val_loss: 4.7396 - val_acc: 0.1391

Epoch 00005: saving model to ../EIP/boosted_resnet.ckpt
Epoch 6/20
97/97 [==============================] - 164s 2s/step - loss: 4.3670 - acc: 0.1708 - val_loss: 4.6942 - val_acc: 0.1423
Epoch 7/20
97/97 [==============================] - 272s 3s/step - loss: 4.3226 - acc: 0.1756 - val_loss: 4.6484 - val_acc: 0.1468
Epoch 8/20
97/97 [===================

In [0]:
model = load_model(checkpoint_path)
model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 512,
  validation_data=val_generator,
  validation_steps=10000 // 512,
  epochs=5,
  max_queue_size=512 * 2,
  callbacks=callbacks, verbose=1
)

Epoch 1/5


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:718: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


  2/195 [..............................] - ETA: 21:23 - loss: 3.8095 - acc: 0.2598

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.511883). Check your callbacks.
  % delta_t_median)


195/195 [==============================] - 323s 2s/step - loss: 3.7897 - acc: 0.2427 - val_loss: 4.4081 - val_acc: 0.1733
Epoch 2/5
195/195 [==============================] - 316s 2s/step - loss: 3.7955 - acc: 0.2408 - val_loss: 4.3221 - val_acc: 0.1799
Epoch 3/5
195/195 [==============================] - 240s 1s/step - loss: 3.7810 - acc: 0.2428 - val_loss: 4.3088 - val_acc: 0.1874
Epoch 4/5
195/195 [==============================] - 151s 776ms/step - loss: 3.7856 - acc: 0.2394 - val_loss: 4.0179 - val_acc: 0.2218
Epoch 5/5
195/195 [==============================] - 152s 777ms/step - loss: 3.7706 - acc: 0.2424 - val_loss: 4.0519 - val_acc: 0.2128

Epoch 00005: saving model to ../EIP/boosted_resnet.ckpt


### Trained it on 32x32 and 16x16 for 20 epochs each. Will now train it on 32x32 to see if any rich features were learnt

---

### Total epochs = 195

In [0]:
model = load_model(checkpoint_path)
model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 512,
  validation_data=val_generator,
  validation_steps=10000 // 512,
  epochs=20,
  max_queue_size=512 * 2,
  callbacks=callbacks, verbose=1
)

Epoch 1/20


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:718: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


195/195 [==============================] - 388s 2s/step - loss: 3.7403 - acc: 0.2589 - val_loss: 3.6560 - val_acc: 0.2730
Epoch 2/20
195/195 [==============================] - 372s 2s/step - loss: 3.2608 - acc: 0.3474 - val_loss: 3.4880 - val_acc: 0.3019
Epoch 3/20
195/195 [==============================] - 210s 1s/step - loss: 3.1104 - acc: 0.3749 - val_loss: 3.4002 - val_acc: 0.3152
Epoch 4/20
195/195 [==============================] - 195s 999ms/step - loss: 3.0173 - acc: 0.3900 - val_loss: 3.3440 - val_acc: 0.3247
Epoch 5/20
195/195 [==============================] - 195s 1s/step - loss: 2.9491 - acc: 0.4010 - val_loss: 3.3033 - val_acc: 0.3366

Epoch 00005: saving model to ../EIP/boosted_resnet.ckpt
Epoch 6/20
195/195 [==============================] - 106s 542ms/step - loss: 2.8959 - acc: 0.4108 - val_loss: 3.2736 - val_acc: 0.3398
Epoch 7/20
195/195 [==============================] - 188s 964ms/step - loss: 2.8612 - acc: 0.4164 - val_loss: 3.2612 - val_acc: 0.3430
Epoch 8/20
195

### WIll now train on 64x64 images with a small batch size and low learning rate

---

### Total epochs = 215

In [0]:
model = load_model('../EIP/resnet.hdf5')
model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 64,
  validation_data=val_generator,
  validation_steps=10000 // 64,
  epochs=35,
  max_queue_size=128,
  callbacks=callbacks, verbose=1
)

Epoch 1/35


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:718: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


1562/1562 [==============================] - 614s 393ms/step - loss: 3.0140 - acc: 0.4086 - val_loss: 3.0803 - val_acc: 0.4164
Epoch 2/35
1562/1562 [==============================] - 590s 378ms/step - loss: 2.4197 - acc: 0.5253 - val_loss: 2.8674 - val_acc: 0.4519
Epoch 3/35
1562/1562 [==============================] - 588s 376ms/step - loss: 2.2614 - acc: 0.5617 - val_loss: 2.7715 - val_acc: 0.4695
Epoch 4/35
1562/1562 [==============================] - 595s 381ms/step - loss: 2.1726 - acc: 0.5823 - val_loss: 2.7162 - val_acc: 0.4788
Epoch 5/35
1562/1562 [==============================] - 604s 387ms/step - loss: 2.1176 - acc: 0.5936 - val_loss: 2.6835 - val_acc: 0.4895

Epoch 00005: saving model to ../EIP/boosted_resnet.ckpt
Epoch 6/35
1562/1562 [==============================] - 593s 380ms/step - loss: 2.0769 - acc: 0.6031 - val_loss: 2.6489 - val_acc: 0.4943
Epoch 7/35
1562/1562 [==============================] - 639s 409ms/step - loss: 2.0447 - acc: 0.6108 - val_loss: 2.6406 - val_

### Trained another 30 epochs

---

### Total epochs = 245

In [0]:
model = load_model(checkpoint_path)
model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 64,
  validation_data=val_generator,
  validation_steps=10000 // 64,
  epochs=35,
  max_queue_size=128,
  callbacks=callbacks, verbose=1
)

Epoch 1/35


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:718: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


1562/1562 [==============================] - 631s 404ms/step - loss: 1.8356 - acc: 0.6607 - val_loss: 2.5224 - val_acc: 0.5232
Epoch 2/35
1562/1562 [==============================] - 628s 402ms/step - loss: 1.8350 - acc: 0.6594 - val_loss: 2.5434 - val_acc: 0.5169
Epoch 3/35
1562/1562 [==============================] - 626s 401ms/step - loss: 1.8352 - acc: 0.6596 - val_loss: 2.5512 - val_acc: 0.5206
Epoch 4/35
1562/1562 [==============================] - 623s 399ms/step - loss: 1.8332 - acc: 0.6605 - val_loss: 2.5043 - val_acc: 0.5264
Epoch 5/35
1562/1562 [==============================] - 617s 395ms/step - loss: 1.8250 - acc: 0.6616 - val_loss: 2.5145 - val_acc: 0.5252

Epoch 00005: saving model to ../EIP/boosted_resnet.ckpt
Epoch 6/35
1562/1562 [==============================] - 591s 378ms/step - loss: 1.8208 - acc: 0.6618 - val_loss: 2.4959 - val_acc: 0.5283
Epoch 7/35
1562/1562 [==============================] - 632s 405ms/step - loss: 1.8151 - acc: 0.6650 - val_loss: 2.5205 - val_

In [0]:
from keras.models import load_model
model = load_model(checkpoint_path)
K.set_value(model.optimizer.momentum, 0.99)

model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 64,
  validation_data=val_generator,
  validation_steps=10000 // 64,
  epochs=35,
  max_queue_size=128,
  callbacks=callbacks, verbose=1
)

Epoch 1/35


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:718: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


1562/1562 [==============================] - 649s 415ms/step - loss: 1.7766 - acc: 0.6720 - val_loss: 2.5256 - val_acc: 0.5248
Epoch 2/35
1562/1562 [==============================] - 629s 402ms/step - loss: 1.7622 - acc: 0.6768 - val_loss: 2.5321 - val_acc: 0.5243
Epoch 3/35
1562/1562 [==============================] - 629s 403ms/step - loss: 1.7786 - acc: 0.6719 - val_loss: 2.5092 - val_acc: 0.5281
Epoch 4/35
1562/1562 [==============================] - 627s 401ms/step - loss: 1.7661 - acc: 0.6753 - val_loss: 2.5167 - val_acc: 0.5248

Epoch 00004: saving model to ../EIP/boosted_resnet.ckpt
Epoch 5/35
1562/1562 [==============================] - 591s 379ms/step - loss: 1.7702 - acc: 0.6750 - val_loss: 2.5076 - val_acc: 0.5278
Epoch 6/35
1562/1562 [==============================] - 627s 401ms/step - loss: 1.7681 - acc: 0.6740 - val_loss: 2.5103 - val_acc: 0.5281
Epoch 7/35
1562/1562 [==============================] - 627s 401ms/step - loss: 1.7667 - acc: 0.6758 - val_loss: 2.4962 - val_

In [0]:
model.save("../EIP/main.hdf5")

In [0]:
from keras.models import load_model
model = load_model('../EIP/main.hdf5')
K.set_value(model.optimizer.momentum, 0.99)

model.fit_generator(
  train_generator,
  steps_per_epoch=100000 // 64,
  validation_data=val_generator,
  validation_steps=10000 // 64,
  epochs=35,
  max_queue_size=128,
  callbacks=callbacks, verbose=1
)

Epoch 1/35


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:699: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:718: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


1562/1562 [==============================] - 636s 407ms/step - loss: 1.7555 - acc: 0.6778 - val_loss: 2.4970 - val_acc: 0.5304
Epoch 2/35
1562/1562 [==============================] - 609s 390ms/step - loss: 1.7364 - acc: 0.6821 - val_loss: 2.5131 - val_acc: 0.5267
Epoch 3/35
1562/1562 [==============================] - 607s 389ms/step - loss: 1.6877 - acc: 0.6937 - val_loss: 2.4934 - val_acc: 0.5333
Epoch 4/35
1562/1562 [==============================] - 602s 386ms/step - loss: 1.6705 - acc: 0.6963 - val_loss: 2.5147 - val_acc: 0.5314
Epoch 5/35
1562/1562 [==============================] - 600s 384ms/step - loss: 1.6431 - acc: 0.7030 - val_loss: 2.5274 - val_acc: 0.5308

Epoch 00005: saving model to ../EIP/boosted_resnet.ckpt
Epoch 6/35
1562/1562 [==============================] - 565s 361ms/step - loss: 1.6277 - acc: 0.7045 - val_loss: 2.4906 - val_acc: 0.5360
Epoch 7/35
1562/1562 [==============================] - 603s 386ms/step - loss: 1.6073 - acc: 0.7102 - val_loss: 2.5409 - val_

In [0]:
model.save("../EIP/main2.hdf5")

### List of references for easy lookup

---

1. Building blocks of interpretability: [Link](https://distill.pub/2018/building-blocks/) (Holy Grail of Intuition!)
2. Deep Residual Learning for image classification: [Link](https://arxiv.org/abs/1512.03385) (Resnet Paper)
3. Bag of tricks for image classification: [Link](https://arxiv.org/abs/1812.01187) (Tweaks and tricks to Resnet for increased performance paper)
2. Imbalanced Deep Learning by Minority Class
Incremental Rectification: [Link](https://arxiv.org/pdf/1804.10851.pdf) (Selectively Sampling Data paper)
2. Improved Regularization of Convolutional Neural Networks with Cutout: [Link](https://arxiv.org/pdf/1708.04552.pdf) (Cutout/Occlusion Augmentation paper)
3. Survey of resampling techniques for improving
classification performance in unbalanced datasets [Link](https://arxiv.org/pdf/1608.06048v1.pdf) (Resampling paper)
4. https://arxiv.org/pdf/1804.10851.pdf
5. https://github.com/wpriyadarshani/Resampling_Techniques
6.   https://www.researchgate.net/post/Machine_learning_if_proportion_of_number_of_cases_in_different_class_in_training_set_matters
7. https://arxiv.org/pdf/1801.05365.pdf
8. https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/
9. https://www.analyticsvidhya.com/blog/2017/03/imbalanced-classification-problem/
10. https://arxiv.org/pdf/1608.06048v1.pdf